In [1]:
import requests

api_key = "tEdVoKjbCzRuX1SnQPZyZYVzUlsZSrauLdxmMIWt"
url = f"https://api.nasa.gov/planetary/apod?api_key={api_key}"

response = requests.get(url)
data = response.json()

print(data)

{'date': '2024-02-24', 'explanation': "Intuitive Machines' robotic lander Odysseus has accomplished the first U.S. landing on the Moon since the Apollo 17 mission in 1972. Launched on a SpaceX rocket on February 15, the phone booth sized lander reached lunar orbit on the 21st and touched down on the lunar surface at 6:23 pm ET on February 22nd. Its landing region is about 300 kilometers north of the Moon's south pole, near a crater designated Malapert A. Resting on its side, the lander is presently collecting solar power and transmitting data back to the Intuitive Machines' mission control center in Houston. The mission marks the first commercial uncrewed landing on the Moon. Prior to landing, Odysseus’ camera captured this extreme wide angle image (landing legs visible at right) as it flew over Schomberger crater some 200 kilometers from its landing site. Odysseus was still about 10 kilometers above the lunar surface.", 'hdurl': 'https://apod.nasa.gov/apod/image/2402/im-moon-imageFeb2

In [3]:
url = f"https://api.nasa.gov/EPIC/api/natural?api_key={api_key}"

response = requests.get(url)
images = response.json()

# Print the URL of the latest image
if images:
    image_date = images[0]['date'].split()[0].replace('-', '/')
    image_name = images[0]['image']
    print(f"https://epic.gsfc.nasa.gov/archive/natural/{image_date}/png/{image_name}.png")

https://epic.gsfc.nasa.gov/archive/natural/2024/02/23/png/epic_1b_20240223002712.png
